Script adaptado a partir de https://huggingface.co/blog/how-to-generate


# Ambiente

In [ ]:
import IPython

In [ ]:
!pip install transformers
!pip install tensorflow


# Como gerar texto usando diferentes métodos de decodificação para geração de linguagem com Transformers

Além da arquitetura aprimorada do `Transformer` e da quantidade massiva de dados de treinamento não supervisionados, **métodos de decodificação** também desempenham um papel importante na rápida evolução que presenciamos em PLN.

As funcionalidades a seguir podem ser usadas para geração de linguagem de forma **auto-regressiva**.


Usaremos GPT2 no Tensorflow para demonstração, mas a API é a mesma para PyTorch (sem TF na frente).

Mais detalhes sobre os parâmetros que podem ser usados na geração do texto podem ser encontrados [aqui](https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/text_generation#transformers.GenerationConfig) (clique em expandir parâmetros).

In [ ]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = TFGPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


### **Greedy Search**

A *greedy search* (pesquisa gulosa) simplesmente seleciona a palavra com a maior probabilidade como sua próxima palavra. O esboço a seguir mostra uma pesquisa gulosa.

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)


Começando com a palavra "The", o algoritmo escolhe de forma gulosa a próxima palavra de maior probabilidade "nice" e assim por diante, de modo que a sequência de palavras gerada final seja {"The", "nice", "woman"} tendo uma probabilidade geral de $0.5 \times 0.4 = 0.2$.

A seguir iremos gerar sequências de palavras usando GPT2 no contexto {"I", "enjoy", "walking", "with", "my", "cute", "dog"}. Vamos ver como a busca gulosa pode ser usada com a bilbioteca `transformers` da seguinte forma:



In [ ]:
# condifica o contexto que iremos utilizar:
input_ids = tokenizer.encode('I enjoy walking with my cute dog', return_tensors='tf')

# gera o texto até o tamanho máximo (inclui o tamamho do contexto)
greedy_output = model.generate(input_ids, max_length=50)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll


As palavras geradas seguindo o contexto são razoáveis, mas o modelo rapidamente começa a se repetir! Este é um problema muito comum na geração de linguagem em geral.

A principal desvantagem da pesquisa gulosa é que ela perde palavras de alta probabilidade escondidas atrás de uma palavra de baixa probabilidade, como pode ser visto na figura exemplo inicial:

A palavra "has" com sua alta probabilidade condicional de 0,9 está escondida atrás da palavra "dog", que tem apenas a segunda maior probabilidade condicional, de modo que a pesquisa gulosa perde a sequência de palavras "The", "dog", "has".

Vamos tentar atenuar isso com o *Beam Search* (busca por feixes).

### **Beam search**

*Beam Search* reduz o risco de perder sequências de palavras de alta probabilidade ocultas, mantendo o mais provável `num_beams` de hipóteses em cada passo tentando escolher a hipótese que tem a probabilidade geral mais alta. Vamos ilustrar com `num_beams=2`:

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)

No intervalo de tempo $1$, além da hipótese mais provável "The", "nice", a pesquisa de feixe também rastreia a segunda hipótese mais provável "The", "dog". No intervalo de tempo $2$, a pesquisa de feixe descobre que a sequência de palavras "The", "dog", "has" tem $0.36$ uma probabilidade maior do que "The", "nice", "woman", que tem $0.2$. Assim, ele encontrou a sequência de palavras mais provável do exemplo.

Vamos ver como *beam search* pode ser usada com `transformers`. Definimos `num_beams > 1` e `early_stopping=True` para que a geração seja concluída assim que houver `num_beams` candidatos completos.

In [ ]:
# beam search com early_stopping
beam_output = model.generate(
    input_ids,
    max_length=50,
    # max_new_tokens = 50,
    num_beams=5, # 1 significa sem beam search
    early_stopping=True,
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I'm not sure if I'll ever be able to walk with him again. I'm not sure if I'll


A saída ainda inclui repetições das mesmas sequências de palavras.
Uma solução simples é introduzir penalidades de *n-gramas*, conforme proposto por [Paulus et al. (2017)](https://arxiv.org/abs/1705.04304) e [Klein et al. (2017)](https://arxiv.org/abs/1701.02810). Usando penalidade de *2-gramas* garante que nenhum *2-grama* apareça duas vezes.



In [ ]:
# usando no_repeat_ngram_size
beam_output = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=2, # se > 0, todos os ngrams deste tamanho somente podem ocorrer uma vez .
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to take a break


Podemos ver que a repetição não aparece mais. No entanto, as penalidades de *n-gram* devem ser usadas com cuidado. Por exemplo, gerando um texto sobre a cidade de *São Paulo* o nome da cidade apareceria apenas uma vez em todo o texto!

Outra característica importante sobre *beam search* é que podemos comparar as sequências geradas e escolher o "feixe" (caminho da sequência) gerado que melhor se adapta ao nosso propósito.

Em `transformers`, nós simplesmente definimos o parâmetro `num_return_sequences` e ele vai retornar as sequências de maior pontuação até esse limite. Certifique-se de que `num_return_sequences <= num_beams` (pois no minimo precisa ser a quantidade de "feixes" que vamos gerar na busca).

In [ ]:
# usando return_num_sequences > 1
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=5,
    no_repeat_ngram_size=3,
    num_return_sequences=4,
    early_stopping=True
)

for i, beam_output in enumerate(beam_outputs):
    print(100 * '-')
    print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been doing this for a few years now, and I've never had a problem with it. I've
----------------------------------------------------------------------------------------------------
1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been doing this for a few years now, and I've never had a problem with it. It's
----------------------------------------------------------------------------------------------------
2: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been doing this for a few years now, and I've never had a problem with it. I'm
----------------------------------------------------------------------------------------------------
3: I enjoy walking with my cute dog, but I

Como pode ser visto, as sequências são apenas marginalmente diferentes umas das outras - o que não deve ser muito surpreendente quando se usa poucos feixes (`num_beams`).

Na geração aberta (*open-ended generation*), algumas razões foram apresentadas recentemente porque a busca de feixe pode não ser a melhor opção possível:

- *Beam search* pode funcionar muito bem em tarefas onde o comprimento da geração desejada é mais ou menos previsível, como na tradução automática ou no sumarização - consulte [Murray et al. (2018)](https://arxiv.org/abs/1808.10006) e [Yang et al. (2018)](https://arxiv.org/abs/1808.09582). Mas este não é o caso da geração *open-ended*, onde o comprimento de saída desejado pode variar muito, por exemplo diálogo e geração de histórias.

- Vimos que *beam search* sofre muito com a geração repetitiva. Isso é difícil de controlar com *n-gram*- ou outras penalidades na geração de histórias, pois encontrar um bom equilíbrio entre forçar a não-repetição e repetir ciclos de *n-grams* idênticos requer muito ajuste fino.

- Conforme argumentado em [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751), a linguagem humana de alta qualidade não segue uma distribuição de próximas palavras de alta probabilidade. Em outras palavras, como humanos, queremos que o texto gerado nos surpreenda e não seja chato/previsível. Os autores mostram isso traçando a probabilidade que um modelo daria ao texto humano versus o que o beam search faz.

![alt text](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)


Então, vamos deixar de ser chatos e introduzir um pouco de aleatoriedade 🤪.

### **Sampling**
Em sua forma mais básica, amostragem (*sampling*) significa escolher aleatoriamente a próxima palavra $w_t$ de acordo com sua distribuição de probabilidade condicional:

$$w_t \sim P(w|w_{1:t-1})$$

Tomando o exemplo da primeira figura, o gráfico a seguir visualiza a geração de linguagem durante a amostragem.

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

Podemos ver que a geração de linguagem usando amostragem não é mais *determinística*. A palavra
"car" é amostrada da distribuição de probabilidade condicional $P(w | "The")$, seguido por amostragem "drives" de P(w | "The", "car"). O modelo nao vai selecionar probabilidades muito proximas de zero.

Na biblioteca `transformers`, configuramos `do_sample=True` e desativamos a amostragem *Top-K* (mais sobre isso mais tarde) via `top_k=0`. Definimos `random_seed=0` para fins de reprodutibilidade.

In [ ]:
# lembrando a sequencia de entrada que usamos:
IPython.display.Markdown(tokenizer.decode(input_ids.numpy()[0]))

I enjoy walking with my cute dog

In [ ]:
# seed para reproduzir resultados de mais de uma execução
tf.random.set_seed(0)

# ativa sampling e desativa  top_k (top_k=0)
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0
)

print("Output:\n" + 100 * '-')
IPython.display.Markdown(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------


I enjoy walking with my cute dog Reyn: on the platform in a bowling alley in Brooklyn, mint one," the partner admitted, laughing at the furred hair pounced on her collar. "Really? I just sat down and watched the rainbow bear

O texto parece bom, mas não é muito coerente. Esse é o grande problema ao amostrar sequências de palavras: os modelos geralmente geram passagens incoerentes [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751).

Um truque é tornar a distribuição $P(w|w_{1:t-1})$ mais nítida (aumenta-se a probabilidade de palavras com alta probabilidade e diminui-se de palavras com baixa probabilidade), o que se denomina diminuir a "temperatura" do [softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max).

Uma ilustração da aplicação de temperatura ao nosso exemplo acima:

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

A distribuição condicional da próxima palavra do passo $t=1$ torna-se muito mais nítida, deixando quase nenhuma chance para a palavra "car" ser selecionada.




De forma geral, sobre temperatura:

- temperaturas menores: produzem um texto mais conservador, pois dá destaque a palavras prováveis

- temperaturas maiores: texto mais diverso, pois vai utilizar também palavras com baixas probabilidades, mas não tão baixas


In [ ]:
tf.random.set_seed(0)

# usa temperatura para diminuir a sensibilidade a candidatos com probabilidades mais baixas
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0,
    temperature=0.7
)

print("Output:\n" + 100 * '-')
IPython.display.Markdown(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------


I enjoy walking with my cute dog and I like having a nice view of the world, including the one I'm walking through, but at the same time, I've got a little bit of an anxiety about it. And I'm trying to be

A saída é um pouco mais coerente agora. Embora a aplicação de temperatura possa tornar uma distribuição menos aleatória, em seu limite, ao definir `temperatura` $ \to 0$, a amostragem em escala de temperatura torna-se igual à decodificação gulosa e sofrerá dos mesmos problemas de antes.

In [ ]:
tf.random.set_seed(0)

# usa temperatura para diminuir a sensibilidade a candidatos com probabilidades mais baixas
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=0,
    temperature=0.05
)

print("Output:\n" + 100 * '-')
IPython.display.Markdown(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------


I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure if I'll

### **Top-K Sampling**
[Fan et. al (2018)](https://arxiv.org/pdf/1805.04833.pdf) introduziu um esquema de amostragem simples, mas muito poderoso, chamado de amostragem ***Top-K***. Na amostragem *Top-K*, as *K* próximas palavras mais prováveis são filtradas e a massa de probabilidade é redistribuída apenas entre essas *K* próximas palavras.

Ampliamos o intervalo de palavras usado para ambas as etapas de amostragem no exemplo acima de 3 para 10 palavras para melhor ilustrar a amostragem *Top-K*.

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)

Tendo definido $K = 6$, limitamos nosso pool de amostragem a 6 palavras, definido como  $V_{\text{top-K}}$. Vemos que ele elimina com sucesso os candidatos bastante estranhos "not", "the", "small", "told" na segunda etapa de amostragem.

Vamos testar como *Top-K* pode ser usado:

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

# set top_k to 50
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50
)

print("Output:\n" + 100 * '-')
IPython.display.Markdown(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------


I enjoy walking with my cute dog and I miss having to walk with my dog in the middle of one of the city's busiest hours at dusk. I was also inspired by the idea of creating an interactive map of the city but I think they really

Na etapa $t=1$, *Top-K* elimina a possibilidade de
amostra $\text{"people", "big", "house", "cat"}$, que parecem candidatos razoáveis. Por outro lado, na etapa $t=2$, o método inclui as palavras que não se ajustam bem $\text{"down", "a"}$ no conjunto de palavras de amostra. Assim, limitar o conjunto de amostras a um tamanho fixo *K* pode
limitar a criatividade do modelo (por exemplo, descarte da palavra $\text{"people"}$) ou levar o mesmo a produzir passagens incoerentes com distribuições "aumentadas" (por exemplo, inclusão da palavra $\text{"down"}$).

Essa intuição levou [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751) para criar ***Top-p***- ou ***nucleus***-sampling.

### **Top-p (nucleus) sampling**

Em vez de amostrar apenas das *K* palavras mais prováveis, na amostragem *Top-p* escolhe-se o menor conjunto possível de palavras cuja probabilidade cumulativa exceda a probabilidade *p* (parâmetro informado). A massa de probabilidade é então redistribuída entre este conjunto de palavras. Vamos visualizar:

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

Na amostragem *Top-p*, você define um parâmetro `p` que representa a massa de probabilidade cumulativa que você deseja cobrir.

Tendo definido $p=0.92$, a amostragem *Top-p* escolhe o número ***mínimo*** de palavras que juntas somam ou excedem $p=92\%$, esse conjunto é definido como definida como $V_{\text{top-p}} $. No primeiro exemplo, isso incluiu as 9 palavras mais prováveis, enquanto no segundo exemplo só precisou escolher as 3 palavras principais para exceder 92%.


Para ativar amostragem com *Top-p* usamos `0 < top_p < 1`:



In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

# deactivate top_k sampling and sample only from 92% most likely words
sample_output = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
IPython.display.Markdown(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------


I enjoy walking with my cute dog and how on Earth this actually works, although I was reminded of one particular episode where Captain America comes to Vancouver Island to visit Agent Coulson at his house, for $1.50 to get him to act kind

Embora em teoria *Top-p* pareça mais elegante que *Top-K*, ambos os métodos funcionam bem na prática. *Top-p* também pode ser usado em combinação com *Top-K*, que pode evitar palavras de classificação muito baixa enquanto permite alguma seleção dinâmica.

Por fim, para obter várias saídas amostradas independentemente, podemos *novamente* definir o parâmetro `num_return_sequences > 1`:

In [ ]:
# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=50,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)


In [ ]:
for i, s in enumerate(sample_outputs):
    print(100 * '-')
    print(tokenizer.decode(s, skip_special_tokens=True))


----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog and I like having a fun time, so I was going to stop in the restaurant and get some coffee."

The second person I saw who didn't wear makeup at the restaurant was her ex-boyfriend
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, and the time you spent with my cats was wonderful and truly worth it. I also found it really funny that my dog gets all upset and irritable before he goes to bed, and is even more upset when
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, so it has been such an inspiration to me in her life."

Morton's story was shared on social media.

The mother-of-two said it was about the "trem


### **Conclusão**



Como métodos de decodificação *ad-hoc*, a amostragem *top-p* e *top-K* parecem produzir texto mais fluente do que a pesquisa gulosa (*greedy search*) tradicional - e *beam search* na geração de linguagem aberta (*open-ended*).
Recentemente, evidências tem sido apresentadas de que as falhas aparentes da pesquisa *greedy* e *beam* - principalmente gerando sequências de palavras repetitivas - são causadas pelo modelo (especialmente a forma como o modelo é treinado), e não pelo método de decodificação [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf). Além disso, conforme demonstrado em [Welleck et al. (2020)](https://arxiv.org/abs/2002.02492), parece que a amostragem *top-K* e *top-p* também sofrem com a geração de sequências de palavras repetitivas.

Em [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf), os autores mostram que, de acordo com avaliações humanas, a pesquisa *beam* pode gerar texto mais fluente do que a amostragem *Top-p*, ao adaptar o modelo objetivo do treinamento.

A geração de linguagem aberta é um campo de pesquisa em rápida evolução e, como costuma acontecer, não existe um método único para todos aqui, portanto, é preciso ver o que funciona melhor no caso de uso específico.


## **Outros Parâmetros**
Alguns parâmetros adicionais para o método `generate`:

- `min_length` pode ser usado para forçar o modelo a não produzir um token EOS (= não terminar a frase) antes que `min_length` seja alcançado. Isso é usado com bastante frequência em resumos, mas pode ser útil em geral se o usuário quiser ter saídas mais longas.
- `repetition_penalty` pode ser usado para penalizar palavras que já foram geradas ou pertencem ao contexto. Foi introduzido pela primeira vez por [Kesker et al. (2019)](https://arxiv.org/abs/1909.05858) e também é usado no objetivo de treinamento em [Welleck et al. (2019)](https://arxiv.org/pdf/1908.04319.pdf). Pode ser bastante eficaz na prevenção de repetições, mas parece ser muito sensível a diferentes modelos e casos de uso, veja esta [discussão](https://github.com/huggingface/transformers/pull/2303) no Github.

Mais parâmetros [aqui](https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/text_generation#transformers.GenerationConfig) (clique em expandir parâmetros).